In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.layers import LSTM, GRU, SimpleRNN
from keras import backend as K


In [2]:
data=pd.read_csv('data/my.csv', index_col=None)
traindata = data.sample(frac=0.7)
testdata =data[~data.index.isin(traindata.index)]

traindata.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,25,0.17.1,0.03,0.17.2,0.24,0.25,0.26,0.05,0.27,normal
48084,0,tcp,systat,S0,0,0,0,0,0,0,...,22,0.09,0.05,0.0,0.00,1.0,1.0,0.0,0.0,neptune
97553,3961,tcp,other,RSTR,1,0,0,0,0,0,...,2,0.01,0.69,1.0,0.00,0.0,0.0,1.0,1.0,portsweep
15289,0,tcp,http,REJ,0,0,0,0,0,0,...,120,1.00,0.00,1.0,0.22,0.0,0.0,1.0,1.0,normal
57769,0,tcp,whois,S0,0,0,0,0,0,0,...,5,0.02,0.08,0.0,0.00,1.0,1.0,0.0,0.0,neptune
14072,0,icmp,eco_i,SF,8,0,0,0,0,0,...,5,1.00,0.00,1.0,0.40,0.0,0.0,0.0,0.0,nmap


In [3]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in traindata.columns:
    #print df[c].dtype
    if traindata[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        traindata[c] = encodings[c].fit_transform(traindata[c])

In [4]:
encodings1 = dict()
for c in testdata.columns:
    #print df[c].dtype
    if testdata[c].dtype == "object":
        encodings1[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings1[c]
        testdata[c] = encodings1[c].fit_transform(testdata[c])

<ipython-input-4-0a51b2024455>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata[c] = encodings1[c].fit_transform(testdata[c])


In [5]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]
Y.head()

48084     9
97553    15
15289    11
57769     9
14072    10
Name: normal, dtype: int32

In [6]:
X=X.drop([X.columns[6],X.columns[14],X.columns[19],X.columns[20],X.columns[8],X.columns[10],X.columns[16],X.columns[17],X.columns[30]], axis = 1)
T=T.drop([X.columns[6],X.columns[14],X.columns[19],X.columns[20],X.columns[8],X.columns[10],X.columns[16],X.columns[17],X.columns[30]], axis = 1)

In [7]:
T.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.4,0.5,0.6,...,0.23,150,25,0.17.1,0.03,0.17.2,0.24,0.25,0.26,0.27
6,0,1,47,5,0,0,0,0,0,0,...,0.0,255,15,0.06,0.07,0.0,0.0,1.0,1.0,0.0
9,0,1,47,1,0,0,0,0,0,0,...,0.0,255,12,0.05,0.07,0.0,0.0,0.0,0.0,1.0
10,0,1,47,5,0,0,0,0,0,0,...,0.0,255,13,0.05,0.07,0.0,0.0,1.0,1.0,0.0
13,0,1,34,5,0,0,0,0,0,0,...,0.0,255,1,0.00,0.07,0.0,0.0,1.0,1.0,0.0
16,0,0,14,9,18,0,0,0,0,0,...,0.0,1,16,1.00,0.00,1.0,1.0,0.0,0.0,0.0


In [8]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
print(trainX[0:5,:])

[[0.000e+00 2.636e-03 1.529e-01 1.318e-02 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  7.196e-01 5.799e-02 2.636e-03 2.636e-03 0.000e+00 0.000e+00 2.109e-04
  1.318e-04 6.722e-01 5.799e-02 2.372e-04 1.318e-04 0.000e+00 0.000e+00
  2.636e-03 2.636e-03 0.000e+00 0.000e+00]
 [9.979e-01 2.519e-04 1.083e-02 1.008e-03 2.519e-04 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  5.038e-04 5.038e-04 0.000e+00 0.000e+00 2.519e-04 2.519e-04 2.519e-04
  0.000e+00 6.424e-02 5.038e-04 2.519e-06 1.738e-04 2.519e-04 0.000e+00
  0.000e+00 0.000e+00 2.519e-04 2.519e-04]
 [0.000e+00 8.168e-03 1.960e-01 8.168e-03 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  8.168e-03 8.168e-03 0.000e+00 0.000e+00 8.168e-03 8.168e-03 8.168e-03
  0.000e+00 8.168e-03 9.802e-01 8.168e-03 0.000e+00 8.168e-03 1.797e-03
  0.000e+00 0.000e+00 8.168e-03 8.168e-03]
 [0.000

In [9]:
scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

In [10]:
y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1,num_classes=23)

In [11]:

# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))
batch_size = 64

In [12]:
X_train.shape

(88180, 32, 1)

In [13]:

cnn = Sequential()
cnn.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(32, 1)))
cnn.add(Convolution1D(64, 3, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(23, activation="softmax"))



In [14]:
lstm_output_size = 70

cnn1 = Sequential()
cnn1.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(32, 1)))
cnn1.add(Convolution1D(64, 3, padding="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_size=(2)))
cnn1.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn1.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_size=(2)))
cnn1.add(Convolution1D(256, 3, padding="same", activation="relu"))
cnn1.add(Convolution1D(256, 3, padding="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_size=(2)))
cnn1.add(LSTM(lstm_output_size))
cnn1.add(Dropout(0.1))
cnn1.add(Dense(23, activation="softmax"))


In [15]:
# try using different optimizers and different optimizer configs
import time
start = time.time()
import tensorflow as tf
cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
#csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
cnn.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))#,callbacks=[checkpointer,csv_logger])
#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")

#loss, accuracy = model.evaluate(X_test, y_test)
#print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
#y_pred = model.predict_classes(X_test)
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', np.transpose([y_test1,y_pred]), fmt='%01d')
end = time.time()
print('exec time = '+str(end-start))


Epoch 1/10
1378/1378 [==============================] - 8s 5ms/step - loss: 0.6965 - accuracy: 0.8382 - precision: 0.8828 - recall: 0.7566 - val_loss: 4.7320 - val_accuracy: 0.5330 - val_precision: 0.5376 - val_recall: 0.5293